In [1]:
#!conda install -c conda-forge googlemaps
#!pip install googlemaps
#!conda install -c conda-forge folium=0.5.0 --yes
!pip install folium

In [2]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import requests

import folium # Map plotting library

from pandas.io.json import json_normalize # Tranform JSON file into a pandas dataframe
import json

# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline

# Import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

import re

### Get all location in Delhi

In [3]:
source = requests.get('https://en.wikipedia.org/wiki/Neighbourhoods_of_Delhi').text 
soup = BeautifulSoup(source, 'lxml')

In [4]:
neighborhood=[]
for i in soup.find_all("h2"):
    for k in i.next:
        flag=0
        if "".join(list(k)) in ['See also','References','External links']:
            flag=1
            break
    if flag==1:
        continue
    for j in i.next_siblings:
        if j.name=='ul':
            text=j.text
            delimiters = ",", ".", "[",";"
            regexPattern = '|'.join(map(re.escape, delimiters))
            for m in text.split('\n'):
                neighborhood.append(re.split(regexPattern,m)[0])
            break

In [5]:
df = pd.DataFrame()
df['Location']=neighborhood

In [6]:
df.head()

,Location
0,Adarsh Nagar
1,Ashok Vihar
2,Begum Pur
3,Karala
4,Model Town


### Get the coordinates of each loaction

In [7]:
# define the rows to get the data
df['Latitude']=None
df['Longitude']=None

In [8]:
def coordinate_extract(soup):
    coor=[]
    search = soup.find_all(class_='BNeawe s3v9rd AP7Wnd')
    for i in search:
        try:
            c=float(i.text)
            coor.append(c)
        except:
            continue
    if len(coor)<2:
        try:
            for i in map(lambda x: re.findall(r'[0-9]{2}\.[0-9]{6}',x.text),search):
                coor.extend(list(map(lambda x: float(x), i)))
            if len(coor)<2: 
                raise exception('')
        except:
            for i in map(lambda x: re.findall(r'[0-9]{2}\.[0-9]{2,6}',x.text),search):
                coor.extend(list(map(lambda x: float(x), i)))
        finally:
            coor=coor[:2]
            
    return coor[0],coor[1]

In [ ]:
for i in df['Location']:
    
    name='+'.join(i.split())
    
    url="https://www.google.com/search?q=delhi {} coordinates".format(name) # Google search at each iterations
    source_code = requests.get(url).text
    soup = BeautifulSoup(source_code, 'lxml')
    
    df.loc[df['Location']==i,['Latitude','Longitude']] = coordinate_extract(soup) # Updating the table

In [ ]:
df.head(3)

### Map the locations using folium

In [ ]:
# Delhi latitude and longitude using Google search
del_lat = 28.7041
del_lng = 77.1025

In [ ]:
# Creates map of Delhi using latitude and longitude values
map_delhi = folium.Map(location=[del_lat, del_lng], zoom_start=11)

# Add markers to map
for lat, lng, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Location']):
    label = '{}'.format(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_delhi)  

In [ ]:
map_delhi

### Use foursquare api to get venue data

**Credentials**

CLIENT_ID, CLIENT_SECRET, VERSION

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
def get_category_type(row):
    categories_list = row['Category']  
    
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
explore_df_list = []

for i, nbd_name in enumerate(df['Location']):  
    
    try :
        # Getting the data of Neighbours
        nbd_name = df.loc[i, 'Location']
        nbd_lat = df.loc[i, 'Latitude']
        nbd_lng = df.loc[i, 'Longitude']

        radius = 1000 # Setting the radius as 1000 metres
        LIMIT = 50 # Getting the top 30 venues

        url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'\
        .format(CLIENT_ID, CLIENT_SECRET, nbd_lat, nbd_lng, VERSION, radius, LIMIT)

        results = json.loads(requests.get(url).text)
        results = results['response']['groups'][0]['items']

        nearby = json_normalize(results) # Flattens JSON

        # Filtering the columns
        filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
        nearby = nearby.loc[:, filtered_columns]

        # Renaming the columns
        columns = ['Name', 'Category', 'Latitude', 'Longitude']
        nearby.columns = columns

        # Gets the categories
        nearby['Category'] = nearby.apply(get_category_type, axis=1)    

        # Gets the data required
        for i, name in enumerate(nearby['Name']):
            s_list = nearby.loc[i, :].values.tolist()  # Converts the numpy array to a python list
            f_list = [nbd_name, nbd_lat, nbd_lng] + s_list
            explore_df_list.append(f_list)
    
    except Exception as e:
        pass

In [ ]:
explore_df = pd.DataFrame([item for item in explore_df_list])
explore_df.columns = ['Location', 'Location Latitude', 'Location Longitude', 'Venue Name', 'Venue Category', 'Venue Latitude', 'Venue Longitude']

In [ ]:
explore_df.head(10)

### Seprate the data with accordance of place where we can get food 

In [ ]:
#all the places where one can get some food

meals=['Restaurant','Place','Food','Joint','Breakfast','Hotel','Coffee','Donut','Dessert','Bakery','Diner','Café','Ice Cream','Cafeteria']

data=explore_df[(explore_df['Venue Category']).apply(lambda x: True if any(item in x.split() for item in meals) else False)==True ]

data.reset_index(inplace=True,drop=True)

data.head(10)

In [ ]:
map_delhi = folium.Map(location=[del_lat, del_lng], zoom_start=11)

# Add markers to map
for lat, lng, Location in zip(data['Venue Latitude'], data['Venue Longitude'], data['Venue Name']):
    label = '{}'.format(Location)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_delhi) 

In [ ]:
map_delhi

### Finding the nearest venues

In [ ]:
# Example coordinate
a1=28.6304
a2=77.2177

In [ ]:
from geopy.distance import distance
place={}
for i in zip(data['Venue Name'],data['Venue Latitude'],data['Venue Longitude']):
    coor=(i[1],i[2])
    k=distance(coor,(a1,a2))
    if k<=2.000000000000000:
        place[i[0]]=[k,i[1],i[2]]
        
places={k: v for k, v in sorted(place.items(), key=lambda item: item[1][0])}

In [ ]:
nearby_venue_latitude=[]
nearby_venue_longitude=[]

for i in places.items():
    nearby_venue_latitude.append(i[1][1])
    nearby_venue_longitude.append(i[1][2])

In [ ]:
final=pd.DataFrame()
final['Name']=places.keys()
final['Latitude']=nearby_venue_latitude
final['Longitude']=nearby_venue_longitude

In [ ]:
final.head()

In [ ]:
map_nearby = folium.Map(location=[a1, a2], zoom_start=14)

folium.Marker([a1,a2]).add_to(map_nearby)

for lat, lng, Location in zip(final['Latitude'], final['Longitude'], final['Name']):
    label = '{}'.format(Location)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(map_nearby) 

In [ ]:
map_nearby